## Trabalho 1 - Reconhecimento de Padrões
Aluno: Luis Felipe Carneiro de Souza
Matrícula: 535049

> Os códigos deste trabalho foram desenvolvido com bases no matarial em MATLAB/OCTAVE disponibilizado pelo professor

In [1]:
import pandas as pd
import numpy as np
import time
from scipy.spatial.distance import minkowski
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
conjunto = pd.read_csv('column_3C.dat', header=None, sep='\s+')
#print(conjunto.info)
conjunto = conjunto.sample(frac=1)
conjunto = conjunto.iloc[:200, :] # Pegando os primeiros 60 elementos para os códigos executarem mais rápido
conjunto.head()

,0,1,2,3,4,5,6
2,68.83,22.22,50.09,46.61,105.99,-3.53,DH
210,38.51,16.96,35.11,21.54,127.63,7.99,NO
106,65.01,9.84,57.74,55.18,94.74,49.70,SL
42,53.85,19.23,32.78,34.62,121.67,5.33,DH
125,48.06,5.69,57.06,42.37,95.44,32.84,SL


In [3]:
x = conjunto.iloc[:, 0:-1]
y = conjunto.iloc[:, -1]

In [11]:
x = conjunto.iloc[:, 0:-1]
y = conjunto.iloc[:, -1]

x_n = normalize(x, norm='l2', axis=1)
x = pd.DataFrame(x_n, index=x.index, columns=x.columns)

In [6]:
sep = {'20/80': {'train': 0.2, 'test': 0.8},
       '30/70': {'train': 0.3, 'test': 0.7},
       '50/50': {'train': 0.5, 'test': 0.5},
       '70/30': {'train': 0.7, 'test': 0.3},
       '80/20': {'train': 0.8, 'test': 0.2}}

### Classificador Vizinho Mais Próximo (distância de Minkowski de ordens m ∈ {0,5; 2/3; 1; 3/2; 2; 5/2})

In [ ]:
resultados_knn = {}
for s in sep.keys():
    resultados_knn[s] = {}

    N_trn = round(sep[s]['train'] * len(conjunto)) # número de amostras do conjunto de treino

    for m in [0.5, 2/3, 1, 3/2, 2, 5/2]: # diferentes ordens da distância de Minkowski

        print(f'\n\nSeparação {s} e distância de Minkowski de ordem {m}\n')

        Pacerto = [] # lista que armazena a porcentagem de acertos em cada rodada
        inicio = time.perf_counter()

        for rep in range(100): # 100 rodadas de treino
            #print(f'Rodada de número {rep + 1}')

            x = x.sample(frac=1)
            y = y.loc[x.index]

            # conjunto de treino
            x_trn, y_trn = x.iloc[:N_trn, :], y.iloc[:N_trn]  
            
            # conjunto de teste
            x_tst, y_tst = x.iloc[N_trn:, :], y.iloc[N_trn:]

            # loop que itera sobre o conjunto de teste
            acerto = 0 # contador de acertos
            for i in range(len(x_tst)):
                x_new = np.array(x_tst.iloc[i, :]) # nova amostra a ser classificada
                y_new = y_tst.iloc[i] # classe da nova amostra

                dist_array = []
                for j in range(len(x_trn)):
                    dist_array.append(minkowski(x_new, np.array(x_trn.iloc[j, :]), p=m)) # distância de Minkowski de ordem m
                
                j_min = np.argmin(dist_array) # índice do menor valor de distância

                if y_trn.iloc[j_min] == y_new: # verifica se a classificação foi correta
                    acerto += 1
        
            Pacerto.append(acerto / len(x_tst)) # calcula a porcentagem de acertos
        fim = time.perf_counter()

        resultados_knn[s][m] = {'media': np.mean(Pacerto),
                           'desvio': np.std(Pacerto),
                           'mediana': np.median(Pacerto),
                           'minimo': np.min(Pacerto),
                           'maximo': np.max(Pacerto),
                           'tempo': fim - inicio}
resultados_knn

In [8]:
resultados_knn

{'20/80': {0.5: {'media': np.float64(0.634375),
   'desvio': np.float64(0.07724504433942672),
   'mediana': np.float64(0.6458333333333334),
   'minimo': np.float64(0.4166666666666667),
   'maximo': np.float64(0.7916666666666666),
   'tempo': 7.105415700003505},
  0.6666666666666666: {'media': np.float64(0.6610416666666665),
   'desvio': np.float64(0.07970513149171206),
   'mediana': np.float64(0.6666666666666666),
   'minimo': np.float64(0.4791666666666667),
   'maximo': np.float64(0.8541666666666666),
   'tempo': 6.0222930000163615},
  1: {'media': np.float64(0.6774999999999999),
   'desvio': np.float64(0.07747535450422642),
   'mediana': np.float64(0.6875),
   'minimo': np.float64(0.3958333333333333),
   'maximo': np.float64(0.8125),
   'tempo': 5.2996325000422075},
  1.5: {'media': np.float64(0.6964583333333334),
   'desvio': np.float64(0.07103250185263393),
   'mediana': np.float64(0.7083333333333334),
   'minimo': np.float64(0.5208333333333334),
   'maximo': np.float64(0.833333333

### Classificador Distância Mínima ao Centróide

In [12]:
resultados_centroide = {}
C = conjunto.iloc[:, -1].unique()
for s in sep.keys():

    #print(f'\n\nSeparação {s}')

    N_trn = round(sep[s]['train'] * len(conjunto)) # número de amostras do conjunto de treino

    Pacerto = [] # lista que armazena a porcentagem de acertos em cada rodada
    inicio = time.perf_counter()

    for rep in range(100): # 100 rodadas de treino
        #print(f'Rodada de número {rep + 1}')

        x = x.sample(frac=1)
        y = y.loc[x.index]
        
        # conjunto de treino
        x_trn, y_trn = x.iloc[:N_trn, :], y.iloc[:N_trn]  
        
        # conjunto de teste
        x_tst, y_tst = x.iloc[N_trn:, :], y.iloc[N_trn:]

        centroides = {}
        for classe in C:
            df_classe = x_trn[y_trn == classe]
            centroides[classe] = df_classe.mean()

        acerto = 0 # contador de acertos
        for i in range(len(x_tst)):
            x_new = np.array(x_tst.iloc[i, :]) # nova amostra a ser classificada
            y_new = y_tst.iloc[i] # classe da nova amostra

            dist_array = []
            for k in C:
                #print(x_new, np.array(centroides[k]))
                dist_array.append(minkowski(x_new, np.array(centroides[k]), p=2)) # distância Euclidiana ao centróide da classe k
            
            k_min = np.argmin(dist_array) # índice do menor valor de distância

            if C[k_min] == y_new:
                acerto += 1 # verifica se a classificação foi correta
        
        Pacerto.append(acerto / len(x_tst))

    fim = time.perf_counter()

    resultados_centroide[s] = {'media': np.mean(Pacerto),
                    'desvio': np.std(Pacerto),
                    'mediana': np.median(Pacerto),
                    'minimo': np.min(Pacerto),
                    'maximo': np.max(Pacerto),
                    'tempo': fim - inicio}
resultados_centroide

{'20/80': {'media': np.float64(0.7438125),
  'desvio': np.float64(0.034938506246690045),
  'mediana': np.float64(0.75),
  'minimo': np.float64(0.64375),
  'maximo': np.float64(0.8125),
  'tempo': 3.5176437000045553},
 '30/70': {'media': np.float64(0.750857142857143),
  'desvio': np.float64(0.04016903061324934),
  'mediana': np.float64(0.75),
  'minimo': np.float64(0.6357142857142857),
  'maximo': np.float64(0.8357142857142857),
  'tempo': 4.050266100093722},
 '50/50': {'media': np.float64(0.7668),
  'desvio': np.float64(0.03515337821604064),
  'mediana': np.float64(0.77),
  'minimo': np.float64(0.68),
  'maximo': np.float64(0.87),
  'tempo': 3.588260800112039},
 '70/30': {'media': np.float64(0.7721666666666667),
  'desvio': np.float64(0.05169649676503989),
  'mediana': np.float64(0.7666666666666667),
  'minimo': np.float64(0.6666666666666666),
  'maximo': np.float64(0.9333333333333333),
  'tempo': 1.3713431000942364},
 '80/20': {'media': np.float64(0.7774999999999999),
  'desvio': np.f

### Classificador Distância Mínima ao Centróide (versão robusta a outliers)

In [16]:
resultados_centroide_robusto = {}
C = conjunto.iloc[:, -1].unique()
for s in sep.keys():

    #print(f'\n\nSeparação {s}')

    N_trn = round(sep[s]['train'] * len(conjunto)) # número de amostras do conjunto de treino

    Pacerto = [] # lista que armazena a porcentagem de acertos em cada rodada
    inicio = time.perf_counter()

    for rep in range(100): # 100 rodadas de treino
        #print(f'Rodada de número {rep + 1}')

        x = x.sample(frac=1)
        y = y.loc[x.index]
        
        # conjunto de treino
        x_trn, y_trn = x.iloc[:N_trn, :], y.iloc[:N_trn]  
        
        # conjunto de teste
        x_tst, y_tst = x.iloc[N_trn:, :], y.iloc[N_trn:]

        centroides = {}
        for classe in C:
            df_classe = x_trn[y_trn == classe]
            centroides[classe] = df_classe.median()

        acerto = 0 # contador de acertos
        for i in range(len(x_tst)):
            x_new = np.array(x_tst.iloc[i, :]) # nova amostra a ser classificada
            y_new = y_tst.iloc[i] # classe da nova amostra

            dist_array = []
            for k in C:
                #print(x_new, np.array(centroides[k]))
                dist_array.append(minkowski(x_new, np.array(centroides[k]), p=1)) # distância quarteirao ao centróide da classe k
            
            k_min = np.argmin(dist_array) # índice do menor valor de distância

            if C[k_min] == y_new:
                acerto += 1 # verifica se a classificação foi correta
        
        Pacerto.append(acerto / len(x_tst))

    fim = time.perf_counter()

    resultados_centroide_robusto[s] = {'media': np.mean(Pacerto),
                    'desvio': np.std(Pacerto),
                    'mediana': np.median(Pacerto),
                    'minimo': np.min(Pacerto),
                    'maximo': np.max(Pacerto),
                    'tempo': fim - inicio}
resultados_centroide_robusto

{'20/80': {'media': np.float64(0.7180000000000001),
  'desvio': np.float64(0.03538052077061614),
  'mediana': np.float64(0.725),
  'minimo': np.float64(0.64375),
  'maximo': np.float64(0.81875),
  'tempo': 3.5181381000438705},
 '30/70': {'media': np.float64(0.7216428571428573),
  'desvio': np.float64(0.03594858857324113),
  'mediana': np.float64(0.7214285714285714),
  'minimo': np.float64(0.6428571428571429),
  'maximo': np.float64(0.8142857142857143),
  'tempo': 2.79889680002816},
 '50/50': {'media': np.float64(0.7329000000000001),
  'desvio': np.float64(0.04336576991130217),
  'mediana': np.float64(0.74),
  'minimo': np.float64(0.59),
  'maximo': np.float64(0.81),
  'tempo': 2.06932680006139},
 '70/30': {'media': np.float64(0.7381666666666667),
  'desvio': np.float64(0.04984949570679952),
  'mediana': np.float64(0.7333333333333333),
  'minimo': np.float64(0.6),
  'maximo': np.float64(0.8666666666666667),
  'tempo': 1.2797238000202924},
 '80/20': {'media': np.float64(0.74025),
  'desv

### Classificador de Máxima Correlação

In [17]:
resultados_similaridade = {}
C = conjunto.iloc[:, -1].unique()

x = conjunto.iloc[:, 0:-1]
y = conjunto.iloc[:, -1]

x_n = normalize(x, norm='l2', axis=1)
x = pd.DataFrame(x_n, index=x.index, columns=x.columns)

for s in sep.keys():

    #print(f'\n\nSeparação {s}')

    N_trn = round(sep[s]['train'] * len(conjunto)) # número de amostras do conjunto de treino

    Pacerto = [] # lista que armazena a porcentagem de acertos em cada rodada
    inicio = time.perf_counter()

    for rep in range(100): # 100 rodadas de treino
        #print(f'Rodada de número {rep + 1}')

        x = x.sample(frac=1)
        y = y.loc[x.index]
        
        # conjunto de treino
        x_trn, y_trn = x.iloc[:N_trn, :], y.iloc[:N_trn]  
        
        # conjunto de teste
        x_tst, y_tst = x.iloc[N_trn:, :], y.iloc[N_trn:]

        centroides = {}
        for classe in C:
            df_classe = x_trn[y_trn == classe]
            centroides[classe] = df_classe.mean()

        acerto = 0 # contador de acertos
        for i in x_tst.index:
            x_new = np.array(x_tst.loc[i, :]) # nova amostra a ser classificada
            y_new = y_tst.loc[i] # classe da nova amostra

            sim = []
            for k in C:
                sim.append(np.dot(x_new, np.array(centroides[k]))) # distância Euclidiana ao centróide da classe k
            
            k_min = np.argmax(sim) # índice do menor valor de distância

            if C[k_min] == y_new:
                acerto += 1 # verifica se a classificação foi correta
        
        Pacerto.append(acerto / len(x_tst))

    fim = time.perf_counter()

    resultados_similaridade[s] = {'media': np.mean(Pacerto),
                    'desvio': np.std(Pacerto),
                    'mediana': np.median(Pacerto),
                    'minimo': np.min(Pacerto),
                    'maximo': np.max(Pacerto),
                    'tempo': fim - inicio}
resultados_similaridade

{'20/80': {'media': np.float64(0.7173125),
  'desvio': np.float64(0.03701620211137279),
  'mediana': np.float64(0.71875),
  'minimo': np.float64(0.625),
  'maximo': np.float64(0.8),
  'tempo': 2.4837039000121877},
 '30/70': {'media': np.float64(0.7300714285714287),
  'desvio': np.float64(0.0356183712056986),
  'mediana': np.float64(0.7285714285714285),
  'minimo': np.float64(0.6571428571428571),
  'maximo': np.float64(0.8214285714285714),
  'tempo': 2.170031200046651},
 '50/50': {'media': np.float64(0.7369999999999999),
  'desvio': np.float64(0.04114608122288198),
  'mediana': np.float64(0.74),
  'minimo': np.float64(0.64),
  'maximo': np.float64(0.83),
  'tempo': 1.6174911999842152},
 '70/30': {'media': np.float64(0.7368333333333332),
  'desvio': np.float64(0.05012733785071776),
  'mediana': np.float64(0.7333333333333333),
  'minimo': np.float64(0.6),
  'maximo': np.float64(0.85),
  'tempo': 0.9555542999878526},
 '80/20': {'media': np.float64(0.7322500000000001),
  'desvio': np.float6

### Resultados

In [ ]:
# 1. Consolidação dos Dados em um único DataFrame

# Lista para armazenar todos os resultados
dados_grafico = []

# Processa os resultados do k-NN
for separacao, resultados_m in resultados_knn.items():
    for m, metricas in resultados_m.items():
        dados_grafico.append({
            'classificador': 'k-NN',
            'separacao': separacao,
            'proporcao_treino': int(separacao.split('/')[0]),
            'parametro_m': f'{m:.2f}', # Formata o 'm' para ter 2 casas decimais
            **metricas # Adiciona todas as outras métricas (media, desvio, etc.)
        })

# Processa os resultados do Centróide
for separacao, metricas in resultados_centroide.items():
    dados_grafico.append({
        'classificador': 'Centróide',
        'separacao': separacao,
        'proporcao_treino': int(separacao.split('/')[0]),
        'parametro_m': 'N/A',
        **metricas
    })

# Processa os resultados do Centróide Robusto
for separacao, metricas in resultados_centroide_robusto.items():
    dados_grafico.append({
        'classificador': 'Centróide Robusto',
        'separacao': separacao,
        'proporcao_treino': int(separacao.split('/')[0]),
        'parametro_m': 'N/A',
        **metricas
    })

# Processa os resultados de Máxima Similaridade
for separacao, metricas in resultados_similaridade.items():
    dados_grafico.append({
        'classificador': 'Máx. Similaridade',
        'separacao': separacao,
        'proporcao_treino': int(separacao.split('/')[0]),
        'parametro_m': 'N/A',
        **metricas
    })

# Cria o DataFrame final
df_final = pd.DataFrame(dados_grafico)


In [ ]:


# 2. Geração do Gráfico de Barras Completo

# Ordena o DataFrame para o gráfico seguir a ordem dos experimentos
ordem_classificador = ['k-NN', 'Centróide', 'Centróide Robusto', 'Máx. Similaridade']
df_final['classificador'] = pd.Categorical(df_final['classificador'], categories=ordem_classificador, ordered=True)
df_ordenado = df_final.sort_values(['classificador', 'proporcao_treino', 'parametro_m'])

# Cria um rótulo único para cada barra para facilitar a visualização no gráfico
df_ordenado['rotulo'] = df_ordenado['classificador'].astype(str) + ' | ' + \
                        df_ordenado['separacao'].astype(str) + ' | m=' + \
                        df_ordenado['parametro_m'].astype(str)

# Ajusta rótulos para classificadores que não usam o parâmetro 'm'
df_ordenado['rotulo'] = df_ordenado['rotulo'].str.replace('\| m=N/A', '', regex=True)

# Configura o estilo do gráfico
sns.set_theme(style="whitegrid", palette="viridis")

# Plota o gráfico
plt.figure(figsize=(20, 10))
ax = sns.barplot(data=df_ordenado, x='rotulo', y='media')

# Adiciona as barras de erro (desvio padrão)
plt.errorbar(x=df_ordenado['rotulo'], y=df_ordenado['media'],
             yerr=df_ordenado['desvio'], fmt='none', c='black', capsize=3)

plt.title('Comparação Completa da Acurácia Média de Todos os Classificadores e Variações', fontsize=20)
plt.xlabel('Classificador | Separação Treino/Teste | Parâmetro (m)', fontsize=14)
plt.ylabel('Acurácia Média', fontsize=14)

# Rotaciona os rótulos do eixo X para que não se sobreponham
plt.xticks(rotation=90)

# Ajusta o limite do eixo Y para dar mais destaque às diferenças de performance
min_media = df_ordenado['media'].min()
max_media = df_ordenado['media'].max()
#plt.ylim(min_media * 0.9, max_media * 1.05)

plt.tight_layout() # Garante que todos os elementos do gráfico fiquem visíveis
plt.show()

In [25]:
df_final

,classificador,separacao,proporcao_treino,parametro,media,desvio,mediana,minimo,maximo,tempo
0,k-NN,20/80,20,0.50,0.634375,0.077245,0.645833,0.416667,0.791667,7.105416
1,k-NN,20/80,20,0.67,0.661042,0.079705,0.666667,0.479167,0.854167,6.022293
2,k-NN,20/80,20,1.00,0.677500,0.077475,0.687500,0.395833,0.812500,5.299633
3,k-NN,20/80,20,1.50,0.696458,0.071033,0.708333,0.520833,0.833333,5.560366
4,k-NN,20/80,20,2.00,0.705833,0.070850,0.729167,0.437500,0.854167,5.962170
5,k-NN,20/80,20,2.50,0.705833,0.063961,0.708333,0.520833,0.854167,6.986426
6,k-NN,30/70,30,0.50,0.662381,0.072639,0.666667,0.452381,0.809524,8.537143
7,k-NN,30/70,30,0.67,0.683810,0.074160,0.690476,0.523810,0.833333,9.027858
8,k-NN,30/70,30,1.00,0.677381,0.073953,0.690476,0.452381,0.809524,14.340648
9,k-NN,30/70,30,1.50,0.696905,0.073375,0.690476,0.476190,0.857143,13.071026
